In [1]:
%matplotlib inline
import importlib
import numpy as np
import pandas as pd
import json
import utils

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/stacked_lstm_statefull/utils.py'>

In [3]:
dat = pd.read_csv('../songdata.csv', usecols=['text']).drop_duplicates()

In [4]:
dat.head()

,text
0,"Look at her face, it's a wonderful face \nAnd..."
1,"Take it easy with me, please \nTouch me gentl..."
2,I'll never know why I had to go \nWhy I had t...
3,Making somebody happy is a question of give an...
4,Making somebody happy is a question of give an...


In [5]:
dat.loc[:, 'text'] += '(end)'

In [6]:
unique_chars = dat.loc[:, 'text'].apply(lambda x: list(set(x)))

In [7]:
unique_chars = unique_chars.sum()

In [8]:
unique_chars = sorted(np.unique(unique_chars))

In [10]:
vocab_size = len(unique_chars)
vocab_size

76

In [11]:
utils.save_unique_tokens(unique_chars)

In [12]:
word2ind, ind2word = utils.get_index_word_map(unique_chars)

In [10]:
word2ind, ind2word = utils.load_index_word_map()

In [13]:
utils.save_index_word_map(word2ind, ind2word)

In [15]:
cleaned = dat.loc[dat.text.apply(len) == 848, 'text']

In [24]:
cleaned.head()

239     When I was young  \nI never needed anyone  \nA...
754     One day of your life is all I'm askin' girl,  ...
806     I want to hop on the last train in the station...
918     Do you ever feel  \nSo perfect  \nYou're not f...
1645    Roots Natty roots  \nDread Binghi dread  \nI a...
Name: text, dtype: object

In [16]:
tokenised = utils.tokenise_cleaned_data(cleaned, word2ind)

In [25]:
tokenised = np.array(tokenised.tolist())

In [26]:
tokenised[:5]

array([[44, 57, 54, ..., 63, 53,  6],
       [36, 63, 54, ..., 63, 53,  6],
       [30,  1, 72, ..., 63, 53,  6],
       [25, 64,  1, ..., 63, 53,  6],
       [39, 64, 64, ..., 63, 53,  6]])

In [30]:
def scanning(array, input_length):
    batches = []
    text_length = array.shape[1]
    for i in range(text_length-input_length):
        batches.append(array[:, i:i+input_length+1])
    return np.concatenate(batches)

In [31]:
batched = scanning(tokenised, 100)

In [32]:
batched.shape

(59092, 101)

In [37]:
59092/79

748.0

In [33]:
tokenised.shape

(79, 848)

In [34]:
batched[:5, :10]

array([[44, 57, 54, 63,  1, 30,  1, 72, 50, 68],
       [36, 63, 54,  1, 53, 50, 74,  1, 64, 55],
       [30,  1, 72, 50, 63, 69,  1, 69, 64,  1],
       [25, 64,  1, 74, 64, 70,  1, 54, 71, 54],
       [39, 64, 64, 69, 68,  1, 35, 50, 69, 69]])

In [35]:
batched[79:84, :5]

array([[57, 54, 63,  1, 30],
       [63, 54,  1, 53, 50],
       [ 1, 72, 50, 63, 69],
       [64,  1, 74, 64, 70],
       [64, 64, 69, 68,  1]])

In [36]:
pd.DataFrame(batched).to_csv('train.csv', index=False)